In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import show, output_file, figure
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

Using CPS wages first

In [2]:
matched = pd.read_csv('/Users/andersonfrailey/Dropbox/PROD2009_julia.csv')

In [3]:
matched['was'] = matched.JCPS21 + matched.JCPS31

In [4]:
matched.sort_values(by='was', inplace=True)

In [5]:
matched['cumwas'] = np.cumsum(matched['MATCHWT'])
max_cumsum = matched['cumwas'].values[-1]
max_cumsum

164828670.05394235

In [6]:
bin_edges = [0] + list(np.arange(1, (6)) *
                       (max_cumsum / float(5)))

In [7]:
labels = range(1, 6)
matched['bins'] = pd.cut(matched['cumwas'], bins=bin_edges, labels=labels)

In [8]:
max_was_cps = []
for i in range(1,6):
    max_was_cps.append(max(matched['was'][matched['bins'] == i]))
max_was_cps

[0, 6000, 30000, 68000, 2499998]

In [9]:
# Participation Rate for All Tax-Units
ranges = [' 00-20', ' 20-40', ' 40-60', ' 60-80', '80-100']
ssi_rates_cps = []
for i in range(1,6):
    participation = ((matched['MATCHWT'][(matched['SSI'] > 0) & (matched['bins'] == i)]).sum() /
                     (matched['MATCHWT'][matched['bins'] == i]).sum())
    ssi_rates_cps.append(participation)
ssi_rates_cps

[0.06313396472435287,
 0.05055967400405331,
 0.02876841065406896,
 0.021526591787635083,
 0.013573784029032842]

In [10]:
# Participation rate only for filers
ssi_rates_cps_filers = []
for i in range(1,6):
    participation = ((matched['MATCHWT'][(matched['SSI'] > 0) & (matched['bins'] == i) & (matched['FILER'] == 1)]).sum() /
                     (matched['MATCHWT'][(matched['bins'] == i) & (matched['FILER'] == 1)]).sum())
    ssi_rates_cps_filers.append(participation)
ssi_rates_cps_filers

[0.06317076560261892,
 0.045048955979557756,
 0.028628515103854786,
 0.021526591787635083,
 0.013573784029032842]

In [11]:
# Participation rate only for non-filers
ssi_rates_cps_nonfilers = []
for i in range(1,6):
    if i >= 3:
        ssi_rates_cps_nonfilers.append(0.0)
        continue
    participation = ((matched['MATCHWT'][(matched['SSI'] > 0) & (matched['bins'] == i) & (matched['FILER'] == 0)]).sum() /
                     (matched['MATCHWT'][(matched['bins'] == i) & (matched['FILER'] == 0)]).sum())
    ssi_rates_cps_nonfilers.append(participation)
ssi_rates_cps_nonfilers

[0.06308711243240886, 0.07598180623910991, 0.0, 0.0, 0.0]

Using PUF WAS

In [12]:
puf = pd.read_csv('/Users/andersonfrailey/Dropbox/PROD2009_julia.csv')

In [13]:
puf.sort_values(by='e00200', inplace=True)

In [14]:
puf['cumwas'] = np.cumsum(puf['MATCHWT'])
max_cumsum = puf['cumwas'].values[-1]

In [15]:
puf['bins'] = pd.cut(puf['cumwas'], bins=bin_edges, labels=labels)

In [16]:
max_was_puf = []
for i in range(1,6):
    max_was_puf.append(max(puf['e00200'][puf['bins'] == i]))
max_was_puf

[0, 9190, 27000, 56900, 48840000]

In [17]:
ssi_rates_puf = []
for i in range(1,6):
    particpation = (((puf['MATCHWT'][(puf['SSI'] > 0) & (puf['bins'] == i)])).sum() /
                    (puf['MATCHWT'][puf['bins'] == i]).sum())
    ssi_rates_puf.append(particpation)
ssi_rates_puf

[0.05778945471133738,
 0.039776303059364436,
 0.02833740473547202,
 0.02864872877291063,
 0.02301062695369746]

In [18]:
# PUF participation rates using CPS cut offs
cut_offs = [(-9e99, 0), (0, 6000), (6000, 30000), (30000, 68000), (68000, 2499998)]
ssi_rates = []
for tup in cut_offs:
    participation = ((puf['MATCHWT'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1]) & (puf['SSI'] > 0)]).sum() /
                     (puf['MATCHWT'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1])]).sum())
    ssi_rates.append(participation)
ssi_rates

[0.05397539812557065,
 0.04471697397234443,
 0.028029917262772343,
 0.027724562296457325,
 0.021985753243401556]

In [19]:
# PUF participation rates using CPS cut offs - only filers
cut_offs = [(-9e99, 0), (0, 6000), (6000, 30000), (30000, 68000), (68000, 2499998)]
ssi_rates_filers = []
for tup in cut_offs:
    participation = ((puf['MATCHWT'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1]) & (puf['SSI'] > 0) & (puf['FILER'] == 1)]).sum() /
                     (puf['MATCHWT'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1]) & (puf['FILER'] == 1)]).sum())
    ssi_rates_filers.append(participation)
ssi_rates_filers

[0.04325564543600753,
 0.04444916392463162,
 0.028015974078938206,
 0.027724562296457325,
 0.021985753243401556]

In [20]:
# PUF participation rates using CPS cut offs - only non-filers
cut_offs = [(-9e99, 0), (0, 6000), (6000, 30000), (30000, 68000), (68000, 2499998)]
ssi_rates_nonfilers = []
for tup in cut_offs:
    if cut_offs.index(tup) >= 2:
        ssi_rates_nonfilers.append(0.0)
        continue
    participation = ((puf['MATCHWT'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1]) & (puf['SSI'] > 0) & (puf['FILER'] == 0)]).sum() /
                     (puf['MATCHWT'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1]) & (puf['FILER'] == 0)]).sum())
    ssi_rates_nonfilers.append(participation)
ssi_rates_nonfilers

[0.06703083813845637, 0.06113192953231907, 0.0, 0.0, 0.0]

In [21]:
# Max WAS in PUF based on CPS bin levels
max_was_puf_cps = []
for tup in cut_offs:
    max_was_puf_cps.append(max(puf['e00200'][(puf['e00200'] > tup[0]) & (puf['e00200'] <= tup[1])]))
max_was_puf_cps

[0, 6000, 30000, 68000, 2497000]

In [22]:
# Percent of PUF population in each CPS bin
cut_offs = [(-9e99, 0), (0, 6000), (6000, 30000), (30000, 68000), (68000, 2499998)]
pop_puf = []
for tup in cut_offs:
    participation = ((puf['MATCHWT'][(puf['e00200'] <= tup[1])]).sum() /
                     (puf['MATCHWT']).sum())
    pop_puf.append(participation)
pop_puf

[0.2713245154175049,
 0.3587266117553511,
 0.6290643958376608,
 0.8431636535022224,
 0.9998896020183522]

In [23]:
# Pre-Match Data
cps = pd.read_csv('/Users/andersonfrailey/dropbox/BENEFITSTEST_julia.csv')

In [24]:
cps.sort_values(by='WAS', inplace=True)

In [25]:
cps['cumwt'] = np.cumsum(cps['WT'])
max_cumt = cps['cumwt'].values[-1]
cps['bins'] = pd.cut(cps['cumwt'], bins=bin_edges, labels=labels)

In [26]:
max_was_cpsp = []
for tup in cut_offs:
    max_was_cpsp.append(max(cps['WAS'][(cps['WAS'] > tup[0]) & (cps['WAS'] <= tup[1])]))
max_was_cpsp

[0, 6000, 30000, 68000, 2499998]

In [27]:
# Pre-Match CPS participation rates using CPS cut offs
cut_offs = [(-9e99, 0), (0, 6000), (6000, 30000), (30000, 68000), (68000, 2499998)]
ssi_rates_cpsp = []
for tup in cut_offs:
    participation = ((cps['WT'][(cps['WAS'] > tup[0]) & (cps['WAS'] <= tup[1]) & (cps['SSI'] > 0)]).sum() /
                     (cps['WT'][(cps['WAS'] > tup[0]) & (cps['WAS'] <= tup[1])]).sum())
    ssi_rates_cpsp.append(participation)
ssi_rates_cpsp

[0.05919537752090937,
 0.02837102019661809,
 0.02293030440656998,
 0.01911360309456395,
 0.009998016890854266]

In [28]:
# Percent of PUF population in each CPS bin
cut_offs = [(-9e99, 0), (0, 6000), (6000, 30000), (30000, 68000), (68000, 2499998)]
pop_puf = []
for tup in cut_offs:
    participation = ((cps['WT'][(cps['WAS'] <= tup[1])]).sum() /
                     (cps['WT']).sum())
    pop_puf.append(participation)
pop_puf

[0.32013237482099244,
 0.3909195024768247,
 0.5967637320359083,
 0.7925697664696734,
 1.0]

In [29]:
comps = pd.DataFrame({'Max WAS - CPS': max_was_cps,
                      'Max WAS - PUF': max_was_puf_cps,
                      'Max WAS - Pre-Match CPS': max_was_cpsp,
                      'Pre-Match CPS SSI Participation Rate': ssi_rates_cpsp,
                      'SSI Participation Rate - CPS': ssi_rates_cps,
                      'PUF Pop in CPS bins': pop_puf,
                      'SSI Participation Rate - PUF': ssi_rates,
                      'SSI Participation Rate - CPS - Filers': ssi_rates_cps_filers,
                      'SSI Participation Rate - CPS - Non-Filers': ssi_rates_cps_nonfilers,
                      'SSI Participation Rate - PUF - Filers': ssi_rates_filers,
                      'SSI Participation Rate - PUF - Non-Filers': ssi_rates_nonfilers}, index=ranges)
comps

,Max WAS - CPS,Max WAS - PUF,Max WAS - Pre-Match CPS,PUF Pop in CPS bins,Pre-Match CPS SSI Participation Rate,SSI Participation Rate - CPS,SSI Participation Rate - CPS - Filers,SSI Participation Rate - CPS - Non-Filers,SSI Participation Rate - PUF,SSI Participation Rate - PUF - Filers,SSI Participation Rate - PUF - Non-Filers
00-20,0,0,0,0.320132,0.059195,0.063134,0.063171,0.063087,0.053975,0.043256,0.067031
20-40,6000,6000,6000,0.390920,0.028371,0.050560,0.045049,0.075982,0.044717,0.044449,0.061132
40-60,30000,30000,30000,0.596764,0.022930,0.028768,0.028629,0.000000,0.028030,0.028016,0.000000
60-80,68000,68000,68000,0.792570,0.019114,0.021527,0.021527,0.000000,0.027725,0.027725,0.000000
80-100,2499998,2497000,2499998,1.000000,0.009998,0.013574,0.013574,0.000000,0.021986,0.021986,0.000000


In [30]:
# Post Match Benefits
ben_list = ['SSI', 'SNAP', 'SS', 'VB', 'MedicareX', 'MEDICAID']
for item in ben_list:
    total = (puf[item] * puf['MATCHWT']).sum()
    print "{}: {:.2f}".format(item, total)

SSI: 55471573635.60
SNAP: 74974352688.69
SS: 729540546535.17
VB: 147120582355.96
MedicareX: 600272786556.46
MEDICAID: 406493967919.45


In [31]:
for item in ben_list:
    total = (cps[item] * cps['WT']).sum()
    print '{}: {:.2f}'.format(item, total)

SSI: 54344585964.25
SNAP: 71647176986.81
SS: 719089821251.10
VB: 157510501160.46
MedicareX: 593100387827.44
MEDICAID: 395693142121.58


In [32]:
def weighted_mean(pdf, col_name):
    return (float((pdf[col_name] * pdf['wt']).sum()) /
            float(pdf['wt'].sum()))

## Plots using the PUF

In [33]:
participation_df = pd.DataFrame()
participation_df['e00200'] = puf['e00200']
participation_df['wt'] = puf['MATCHWT']
participation_df['SSI Participation'] = np.where(puf['SSI'] > 0, 1, 0)

participation_df.sort_values(by='e00200', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SSI Participation')

means_plot = figure(y_axis_label='SSI Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SSI Participation Rate by PUF Wage Percentile')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

In [34]:
benefit_df = pd.DataFrame()
benefit_df['e00200'] = puf['e00200'][puf['SSI'] > 0]
benefit_df['wt'] = puf['MATCHWT'][puf['SSI'] > 0]
benefit_df['SSI'] = puf['SSI'][puf['SSI'] > 0]

benefit_df.sort_values(by='e00200', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SSI')

benefit_plot = figure(y_axis_label='SSI Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SSI Benefit by PUF Wage Percentile')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [35]:
# Participation
participation_df = pd.DataFrame()
participation_df['e00200'] = puf['e00200']
participation_df['wt'] = puf['MATCHWT']
participation_df['SNAP Participation'] = np.where(puf['SNAP'] > 0, 1, 0)

participation_df.sort_values(by='e00200', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SNAP Participation')

means_plot = figure(y_axis_label='SNAP Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SNAP Participation Rate by PUF Wage Percentile')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['e00200'] = puf['e00200'][puf['SNAP'] > 0]
benefit_df['wt'] = puf['MATCHWT'][puf['SNAP'] > 0]
benefit_df['SNAP'] = puf['SNAP'][puf['SNAP'] > 0]

benefit_df.sort_values(by='e00200', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SNAP')

benefit_plot = figure(y_axis_label='SNAP Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SNAP Benefit by PUF Wage Percentile')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [36]:
# Participation
participation_df = pd.DataFrame()
participation_df['e00200'] = puf['e00200']
participation_df['wt'] = puf['MATCHWT']
participation_df['Medicare Participation'] = np.where(puf['MedicareX'] > 0, 1, 0)

participation_df.sort_values(by='e00200', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'Medicare Participation')

means_plot = figure(y_axis_label='Medicare Participation Rate', x_axis_label='Household Wage Percentile',
                    title='Medicare Participation Rate by PUF Wage Percentile')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['e00200'] = puf['e00200'][puf['MedicareX'] > 0]
benefit_df['wt'] = puf['MATCHWT'][puf['MedicareX'] > 0]
benefit_df['Medicare'] = puf['MedicareX'][puf['MedicareX'] > 0]

benefit_df.sort_values(by='e00200', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'Medicare')

benefit_plot = figure(y_axis_label='Medicare Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean Medicare Benefit by PUF Wage Percentile')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [37]:
# Participation
participation_df = pd.DataFrame()
participation_df['e00200'] = puf['e00200']
participation_df['wt'] = puf['MATCHWT']
participation_df['Medicaid Participation'] = np.where(puf['MEDICAID'] > 0, 1, 0)

participation_df.sort_values(by='e00200', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'Medicaid Participation')

means_plot = figure(y_axis_label='Medicaid Participation Rate', x_axis_label='Household Wage Percentile',
                    title='Medicaid Participation Rate by PUF Wage Percentile')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['e00200'] = puf['e00200'][puf['MEDICAID'] > 0]
benefit_df['wt'] = puf['MATCHWT'][puf['MEDICAID'] > 0]
benefit_df['Medicaid'] = puf['MEDICAID'][puf['MEDICAID'] > 0]

benefit_df.sort_values(by='e00200', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'Medicaid')

benefit_plot = figure(y_axis_label='Medicaid Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean Medicaid Benefit by PUF Wage Percentile')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [38]:
# Participation
participation_df = pd.DataFrame()
participation_df['e00200'] = puf['e00200']
participation_df['wt'] = puf['MATCHWT']
participation_df['VB Participation'] = np.where(puf['VB'] > 0, 1, 0)

participation_df.sort_values(by='e00200', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'VB Participation')

means_plot = figure(y_axis_label='VB Participation Rate', x_axis_label='Household Wage Percentile',
                    title='VB Participation Rate by PUF Wage Percentile')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['e00200'] = puf['e00200'][puf['VB'] > 0]
benefit_df['wt'] = puf['MATCHWT'][puf['VB'] > 0]
benefit_df['VB'] = puf['VB'][puf['VB'] > 0]

benefit_df.sort_values(by='e00200', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'VB')

benefit_plot = figure(y_axis_label='VB Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean VB Benefit by PUF Wage Percentile')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [39]:
# Participation
participation_df = pd.DataFrame()
participation_df['e00200'] = puf['e00200']
participation_df['wt'] = puf['MATCHWT']
participation_df['SS Participation'] = np.where(puf['SS'] > 0, 1, 0)

participation_df.sort_values(by='e00200', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SS Participation')

means_plot = figure(y_axis_label='SS Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SS Participation Rate by PUF Wage Percentile')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['e00200'] = puf['e00200'][puf['SS'] > 0]
benefit_df['wt'] = puf['MATCHWT'][puf['SS'] > 0]
benefit_df['SS'] = puf['SS'][puf['SS'] > 0]

benefit_df.sort_values(by='e00200', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SS')

benefit_plot = figure(y_axis_label='SS Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SS Benefit by PUF Wage Percentile')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

## Plots using the CPS Tax Units

Post-Match, using CPS wages 

In [40]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = matched['WAS']
participation_df['wt'] = matched['MATCHWT']
participation_df['SSI Participation'] = np.where(matched['SSI'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SSI Participation')

means_plot = figure(y_axis_label='SSI Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SSI Participation Rate by CPS Wage Percentile - Post-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = matched['WAS'][matched['SSI'] > 0]
benefit_df['wt'] = matched['MATCHWT'][matched['SSI'] > 0]
benefit_df['SSI'] = matched['SSI'][matched['SSI'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SSI')

benefit_plot = figure(y_axis_label='SSI Benefit', x_axis_label='Household Wage Percentile',
                      title='MEAN SSI Benefit by CPS Wage Percentile - Post-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [41]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = matched['WAS']
participation_df['wt'] = matched['MATCHWT']
participation_df['SNAP Participation'] = np.where(matched['SNAP'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SNAP Participation')

means_plot = figure(y_axis_label='SNAP Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SNAP Participation Rate by CPS Wage Percentile - Post-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = matched['WAS'][matched['SNAP'] > 0]
benefit_df['wt'] = matched['MATCHWT'][matched['SNAP'] > 0]
benefit_df['SNAP'] = matched['SNAP'][matched['SNAP'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SNAP')

benefit_plot = figure(y_axis_label='SNAP Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SNAP Benefit by CPS Wage Percentile - Post-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [42]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = matched['WAS']
participation_df['wt'] = matched['MATCHWT']
participation_df['Medicare Participation'] = np.where(matched['MedicareX'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'Medicare Participation')

means_plot = figure(y_axis_label='Medicare Participation Rate', x_axis_label='Household Wage Percentile',
                    title='Medicare Participation Rate by CPS Wage Percentile - Post-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = matched['WAS'][matched['MedicareX'] > 0]
benefit_df['wt'] = matched['MATCHWT'][matched['MedicareX'] > 0]
benefit_df['Medicare'] = matched['MedicareX'][matched['MedicareX'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'Medicare')

benefit_plot = figure(y_axis_label='Medicare Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean Medicare Benefit by CPS Wage Percentile - Post-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [43]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = matched['WAS']
participation_df['wt'] = matched['MATCHWT']
participation_df['Medicaid Participation'] = np.where(matched['MEDICAID'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'Medicaid Participation')

means_plot = figure(y_axis_label='Medicaid Participation Rate', x_axis_label='Household Wage Percentile',
                    title='Medicaid Participation Rate by CPS Wage Percentile - Post-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = matched['WAS'][matched['MEDICAID'] > 0]
benefit_df['wt'] = matched['MATCHWT'][matched['MEDICAID'] > 0]
benefit_df['Medicaid'] = matched['MEDICAID'][matched['MEDICAID'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'Medicaid')

benefit_plot = figure(y_axis_label='Medicaid Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean Medicaid Benefit by CPS Wage Percentile - Post-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [44]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = matched['WAS']
participation_df['wt'] = matched['MATCHWT']
participation_df['VB Participation'] = np.where(matched['VB'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'VB Participation')

means_plot = figure(y_axis_label='VB Participation Rate', x_axis_label='Household Wage Percentile',
                    title='VB Participation Rate by CPS Wage Percentile - Post-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = matched['WAS'][matched['VB'] > 0]
benefit_df['wt'] = matched['MATCHWT'][matched['VB'] > 0]
benefit_df['VB'] = matched['SNAP'][matched['VB'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'VB')

benefit_plot = figure(y_axis_label='VB Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean VB Benefit by CPS Wage Percentile - Post-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [45]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = matched['WAS']
participation_df['wt'] = matched['MATCHWT']
participation_df['SS Participation'] = np.where(matched['SS'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SS Participation')

means_plot = figure(y_axis_label='SS Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SS Participation Rate by CPS Wage Percentile - Post-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = matched['WAS'][matched['SS'] > 0]
benefit_df['wt'] = matched['MATCHWT'][matched['SS'] > 0]
benefit_df['SS'] = matched['SS'][matched['SS'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SS')

benefit_plot = figure(y_axis_label='SS Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SS Benefit by CPS Wage Percentile - Post-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

## Using CPS Family Units

Pre-match, post CPS tax unit creation

In [46]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = cps['WAS']
participation_df['wt'] = cps['WT']
participation_df['SSI Participation'] = np.where(cps['SSI'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SSI Participation')

means_plot = figure(y_axis_label='SSI Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SSI Participation Rates by CPS Wage - Pre-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = cps['WAS'][cps['SSI'] > 0]
benefit_df['wt'] = cps['WT'][cps['SSI'] > 0]
benefit_df['SSI'] = cps['SSI'][cps['SSI'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SSI')

benefit_plot = figure(y_axis_label='SSI Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SSI Benefit by CPS Wage Percentile - Pre-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [47]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = cps['WAS']
participation_df['wt'] = cps['WT']
participation_df['SNAP Participation'] = np.where(cps['SNAP'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SNAP Participation')

means_plot = figure(y_axis_label='SNAP Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SNAP Participation Rates by CPS Wage - Pre-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = cps['WAS'][cps['SNAP'] > 0]
benefit_df['wt'] = cps['WT'][cps['SNAP'] > 0]
benefit_df['SNAP'] = cps['SNAP'][cps['SNAP'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SNAP')

benefit_plot = figure(y_axis_label='SNAP Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean SNAP Benefit by CPS Wage Percentile - Pre-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [48]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = cps['WAS']
participation_df['wt'] = cps['WT']
participation_df['Medicare Participation'] = np.where(cps['MedicareX'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'Medicare Participation')

means_plot = figure(y_axis_label='Medicare Participation Rate', x_axis_label='Household Wage Percentile',
                    title='Medicare Participation Rates by CPS Wage - Pre-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = cps['WAS'][cps['MedicareX'] > 0]
benefit_df['wt'] = cps['WT'][cps['MedicareX'] > 0]
benefit_df['Medicare'] = cps['MedicareX'][cps['MedicareX'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'Medicare')

benefit_plot = figure(y_axis_label='Medicare Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean Medicare Benefit by CPS Wage Percentile - Pre-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [49]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = cps['WAS']
participation_df['wt'] = cps['WT']
participation_df['Medicaid Participation'] = np.where(cps['MEDICAID'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'Medicaid Participation')

means_plot = figure(y_axis_label='Medicaid Participation Rate', x_axis_label='Household Wage Percentile',
                    title='Medicaid Participation Rates by CPS Wage - Pre-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = cps['WAS'][cps['MEDICAID'] > 0]
benefit_df['wt'] = cps['WT'][cps['MEDICAID'] > 0]
benefit_df['Medicaid'] = cps['MEDICAID'][cps['MEDICAID'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'Medicaid')

benefit_plot = figure(y_axis_label='Medicaid Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean Medicaid Benefit by CPS Wage Percentile - Pre-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [50]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = cps['WAS']
participation_df['wt'] = cps['WT']
participation_df['VB Participation'] = np.where(cps['VB'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'VB Participation')

means_plot = figure(y_axis_label='VB Participation Rate', x_axis_label='Household Wage Percentile',
                    title='VB Participation Rates by CPS Wage - Pre-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = cps['WAS'][cps['VB'] > 0]
benefit_df['wt'] = cps['WT'][cps['VB'] > 0]
benefit_df['VB'] = cps['VB'][cps['VB'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'VB')

benefit_plot = figure(y_axis_label='VB Benefit', x_axis_label='Household Wage Percentile',
                      title='Mean VB Benefit by CPS Wage Percentile - Pre-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)

In [51]:
# Participation
participation_df = pd.DataFrame()
participation_df['WAS'] = cps['WAS']
participation_df['wt'] = cps['WT']
participation_df['SS Participation'] = np.where(cps['SS'] > 0, 1, 0)

participation_df.sort_values(by='WAS', inplace=True)
participation_df['cumsum'] = np.cumsum(participation_df['wt'].values)

max_cumsum = participation_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
participation_df['bins'] = pd.cut(participation_df['cumsum'], bins=bin_edges, labels=None)
participation_df.drop('cumsum', axis=1, inplace=True)
participation = participation_df.groupby('bins', as_index=False)
participation_means = participation.apply(weighted_mean, 'SS Participation')

means_plot = figure(y_axis_label='SS Participation Rate', x_axis_label='Household Wage Percentile',
                    title='SS Participation Rates by CPS Wage - Pre-Match')
means_plot.line(participation_means.index.values, participation_means.values)
show(means_plot)

# Mean benefit graph
benefit_df = pd.DataFrame()
benefit_df['WAS'] = cps['WAS'][cps['SS'] > 0]
benefit_df['wt'] = cps['WT'][cps['SS'] > 0]
benefit_df['SS'] = cps['SS'][cps['SS'] > 0]

benefit_df.sort_values(by='WAS', inplace=True)
benefit_df['cumsum'] = np.cumsum(benefit_df['wt'].values)

max_cumsum = benefit_df['cumsum'].values[-1]
bin_edges = [0] + list(np.arange(1, (101)) *
                       (max_cumsum / float(100)))
benefit_df['bins'] = pd.cut(benefit_df['cumsum'], bins=bin_edges, labels=None)
benefit_df.drop('cumsum', axis=1, inplace=True)
benefit = benefit_df.groupby('bins', as_index=False)
benefit_wages = benefit.apply(weighted_mean, 'SS')

benefit_plot = figure(y_axis_label='SS Benefit', x_axis_label='Household Wage Percentile',
                     title='Mean SS Benefit by CPS Wage Percentile - Pre-Match')
benefit_plot.line(benefit_wages.index.values, benefit_wages.values)
show(benefit_plot)